pytorch resnet网络的bottleneck 特征提取
1. 直观的方法，详见[这里](https://discuss.pytorch.org/t/how-can-l-use-the-pre-trained-resnet-to-extract-feautres-from-my-own-dataset/9008/4)
2. 优雅的方法是使用类函数来获得bottleneck层，详见[这里](https://forums.fast.ai/t/pytorch-best-way-to-get-at-intermediate-layers-in-vgg-and-resnet/5707)

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
model = torchvision.models.resnet18(pretrained=True)

In [3]:
# print (dir(model))
print((model.named_parameters)) #同model.parameters， model.children,只是bound method Module.${name} of ResNet 不同
# print (list(model.state_dict()))

<bound method Module.named_parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [4]:
bottleneck_feature = nn.Sequential((*list(model.children())[:-1]))

In [ ]:
print (bottleneck_feature.parameters)

In [ ]:
my_embedding=Variable
def copy_data(m, i, o):
            my_embedding =o.data.squeeze()
        
model.layer4[0].conv2.register_forward_hook(copy_data)


In [5]:
from PIL import Image
import cv2

In [6]:
img_path='3.jpg'

In [7]:
toTensor = transforms.ToTensor()

def prepare_image(file):
    image = Image.open(file)
    print(image.size)
    # size = (244,244)  #任意大小
    # img = image.resize(size , Image.BILINEAR)
    img = toTensor(image)
    img = img.sub_(0.5).div_(0.5)
    img = img.view(1, *img.size())
    return img
img = prepare_image(img_path)

(700, 394)


In [ ]:
model(img).shape

In [8]:
feature = bottleneck_feature(img)
feature.shape

torch.Size([1, 512, 1, 1])

In [9]:
feature.squeeze().data

tensor([0.8916, 0.9683, 0.8634, 1.0349, 0.8481, 0.7674, 0.8648, 1.0380, 0.9475,
        0.9245, 0.8767, 0.8676, 0.8809, 0.9311, 0.8718, 0.9117, 0.8710, 1.3772,
        0.8684, 0.8540, 0.8693, 1.0411, 0.9136, 0.9373, 0.9276, 0.9305, 0.9564,
        0.9462, 0.8380, 0.8828, 0.8104, 0.8221, 0.8057, 0.8248, 0.8901, 0.8975,
        0.8672, 0.9389, 0.8854, 0.8745, 0.8394, 0.8409, 0.8498, 0.9558, 0.8298,
        0.9022, 0.9087, 1.1222, 0.8818, 0.8251, 0.9145, 0.9441, 0.8581, 0.8835,
        0.9302, 0.8793, 1.0677, 0.9063, 1.0129, 0.9507, 0.8713, 0.8485, 0.9283,
        0.8489, 0.8641, 1.0184, 0.8738, 0.8673, 0.9285, 0.8730, 0.8702, 0.7891,
        0.8495, 0.8091, 0.9184, 0.8480, 0.8120, 0.8730, 0.8894, 0.8823, 0.8747,
        0.9346, 1.0223, 1.1561, 0.8316, 0.9479, 0.9101, 0.8574, 0.9358, 0.8261,
        0.7527, 0.8891, 0.8231, 0.9558, 0.8394, 0.7938, 0.8856, 0.8802, 0.9016,
        0.8664, 0.9691, 1.0150, 0.9257, 0.9472, 0.9089, 0.7865, 0.8724, 0.8968,
        0.7964, 0.8742, 0.8628, 0.8773, 

AdaptiveAvgPool2d函数理解：指定feature map的输出大小

In [10]:
m = nn.AdaptiveAvgPool2d((5,7))
inputs = torch.randn(1, 64, 8, 9)
output = m(inputs)
output.size()

torch.Size([1, 64, 5, 7])

keras(2.2.5版本，tf版本为1.13.1) bottleneck feature特征提取，mobilenet会提示找不到膨胀卷积
1. 直观方法是加载预训练模型时，不加载分类层，详见[这里](https://gist.github.com/takahish/eab7d2923ab30d7c925061da60524db3)
2. 定义模型的输入和输出节点，详见[这里](https://github.com/keras-team/keras/issues/8418)

In [11]:
import numpy as np
import keras
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input

Using TensorFlow backend.


In [13]:
bottneck_feature_ks = InceptionV3(weights='imagenet', include_top=False)

In [14]:
def prepare_image (file):
    im_resized = image.load_img(file, target_size = (224,224))
    img_array = image.img_to_array(im_resized)
    image_array_expanded = np.expand_dims(img_array, axis = 0)
    return image_array_expanded
img = prepare_image(img_path)

In [16]:
features = bottneck_feature_ks.predict(preprocess_input(img))
(features.squeeze()).shape

(5, 5, 2048)

In [17]:
features.flatten().shape

(51200,)